In [2]:
# from zipfile import ZipFile
# file_name = "/Users/chandru/Desktop/Python/python4e/code3/6768/archives.zip"

# with ZipFile(file_name,'r') as zipfile:
#     zipfile.extractall()
#     print('Done')

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [1]:
train_dir = "/Users/chandru/Desktop/Python/python4e/code3/6768/archives/train"
val_dir = "/Users/chandru/Desktop/Python/python4e/code3/6768/archives/test"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

In [ ]:
base_model  = tf.keras.applications.ResNet50(input_shape=(48,48,3),include_top=False,weights='imagenet')

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/50
448/448 [==============================] - 388s 864ms/step - loss: 1.7963 - accuracy: 0.2598 - val_loss: 1.6955 - val_accuracy: 0.3478
Epoch 2/50
448/448 [==============================] - 390s 871ms/step - loss: 1.6175 - accuracy: 0.3702 - val_loss: 1.5283 - val_accuracy: 0.4134
Epoch 3/50
448/448 [==============================] - 390s 870ms/step - loss: 1.5157 - accuracy: 0.4179 - val_loss: 1.4446 - val_accuracy: 0.4501
Epoch 4/50
448/448 [==============================] - 389s 869ms/step - loss: 1.4411 - accuracy: 0.4504 - val_loss: 1.3952 - val_accuracy: 0.4696
Epoch 5/50
448/448 [==============================] - 391s 873ms/step - loss: 1.3790 - accuracy: 0.4766 - val_loss: 1.3260 - val_accuracy: 0.4986
Epoch 6/50
448/448 [==============================] - 394s 879ms/step - loss: 1.3289 - accuracy: 0.4986 - val_loss: 1.2920 - val_accuracy: 0.5116
Epoch 7/50
448/448 [==============================] - 395s 882ms/step - loss: 1.2850 - accuracy: 0.5116 - val_loss: 1.2601 -

In [ ]:
fer_json = emotion_model.to_json()
with open("femotion.json", "w") as json_file:
    json_file.write(fer_json)
emotion_model.save_weights("femweight.h5")